In [16]:
import json
import requests
from config import fed_api_key
from pprint import pprint
import pandas as pd
import numpy as np

In [17]:
pop = ['AUSPOP','HTNPOP','DFWPOP','SATPOP','ELPPOP','MCLPOP']
hpi = ['ATNHPIUS12420Q','ATNHPIUS26420Q','ATNHPIUS19124Q','ATNHPIUS41700Q','ATNHPIUS21340Q','ATNHPIUS32580Q']

In [12]:
#'https://api.stlouisfed.org/fred/series/observations?series_id=WPU0811&api_key=0a60bfbafed5ab7080d559305d53573a&file_type=json'
base_url='https://api.stlouisfed.org/fred/series/observations?series_id='
mid_url = '&api_key='
end_url='&file_type=json'
urls =[]
dummies = []
for x in pop:
    url = base_url + x + mid_url + fed_api_key + end_url
    urls.append(url)

In [14]:
#pull populations and create CSVs
x = 0
dates = []
values = []
msa = []
for url in urls:
    response = requests.get(url)
    data_json = response.json()
    for data in data_json['observations']:
        dates.append(data['date'])
        values.append(data['value'])
        msa.append(pop[x])
    x = x+1
df = pd.DataFrame({
    "MetroArea": msa,
    "date": dates,
    "values": values
})
df.to_csv('Resources/Population.csv')

In [15]:
#Pull Pricing index and create CSVs
urls =[]
for y in hpi:
    url = base_url + y + mid_url + fed_api_key + end_url
    urls.append(url)
x = 0
dates = []
values = []
msa = []
for url in urls:
    response = requests.get(url)
    data_json = response.json()
    for data in data_json['observations']:
        dates.append(data['date'])
        values.append(data['value'])
        msa.append(pop[x])
    x = x+1
df = pd.DataFrame({
    "MetroArea": msa,
    "date": dates,
    "values": values
})
df.to_csv('Resources/HPI.csv')

In [21]:
file = "Resources/ZilloHomeValueIndexZip.csv"
zhvi_df = pd.read_csv(file)
zhvi_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,1996-01-31,...,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31,2021-04-30,2021-05-31,2021-06-30,2021-07-31
0,61639,0,10025,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,193814.0,...,1036773.0,1039292.0,1051475.0,1052389.0,1062227.0,1073321.0,1088262.0,1101458.0,1110816.0,1117997.0
1,84654,1,60657,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,262402.0,...,497434.0,499976.0,502451.0,504692.0,507078.0,508667.0,510851.0,513170.0,516360.0,519384.0
2,61637,2,10023,Zip,NY,NY,New York,New York-Newark-Jersey City,New York County,261664.0,...,1105905.0,1101323.0,1092186.0,1083389.0,1083178.0,1081122.0,1084203.0,1086301.0,1088716.0,1088755.0
3,91982,3,77494,Zip,TX,TX,Katy,Houston-The Woodlands-Sugar Land,Harris County,201649.0,...,345982.0,349035.0,353498.0,358004.0,363402.0,369313.0,376254.0,383839.0,392947.0,403712.0
4,84616,4,60614,Zip,IL,IL,Chicago,Chicago-Naperville-Elgin,Cook County,353678.0,...,635561.0,638519.0,641360.0,643819.0,646973.0,649403.0,652545.0,654806.0,658509.0,661889.0


In [ ]:
zhvi2_df = pd.melt(zhvi_df,id_vars=['RegionName',''])